In [ ]:
from google.colab import drive  # 구글 코랩에서 drive(내 구글 드라이브)를 사용하기 위한 함수
import io
drive.mount('/content/drive')  # 이렇게 하면 drive를 인식을 시키는 작업

In [ ]:
import pandas as pd # 데이터분석 라이브러리로, 행과 열로 이루어진 데이터 객체를 만들어 다룰 수 있음
import numpy as np #  다차원 배열을 쉽게 처리하고  효율적으로 사용할 수 있도록 하는 라이브러리
from datetime import datetime
#abc

In [ ]:
filename = '/content/drive/My Drive/data/bigcorn_data/data3.csv' # (세진이가 준 데이터임) 경로를 지정해준다.
filename
df = pd.read_csv(filename) # xlsx 읽기

In [ ]:
df['방송일시']=df['방송일시'].drop_duplicates() # 중복제거

In [ ]:
df=df[['방송일시','방송끝나는시간','노출(분)']].dropna(axis=0) # 방송일시만 남겨놓는 작업

In [ ]:
df.head

<bound method NDFrame.head of                    방송일시           방송끝나는시간  노출(분)
0       2019-01-01 6:00   2019-01-01 6:20   20.0
2       2019-01-01 6:20   2019-01-01 6:40   20.0
4       2019-01-01 6:40   2019-01-01 7:00   20.0
6       2019-01-01 7:00   2019-01-01 7:20   20.0
7       2019-01-01 7:20   2019-01-01 7:40   20.0
...                 ...               ...    ...
35361  2019-12-31 21:20  2019-12-31 21:40   20.0
35363  2019-12-31 21:40  2019-12-31 22:00   20.0
35365  2019-12-31 22:00  2019-12-31 22:20   20.0
35367  2019-12-31 23:20  2019-12-31 23:40   20.0
35371  2019-12-31 23:40   2020-01-01 0:00   20.0

[20565 rows x 3 columns]>

In [ ]:
start_time =[] # 가져온 데이터에서 방송시작시간과 종료시간 가져오기
end_time =[]
for i in range(0,len(df)):
    for j in range(2):
        if j==0:
            start_time.append(df.values[i][j].split())
        else:
            end_time.append(df.values[i][j].split())

In [ ]:
print(start_time[-1])
print(end_time[-1])

['2019-12-31', '23:40']
['2020-01-01', '0:00']


In [ ]:
filename = '/content/drive/My Drive/data/bigcorn_data/data2.xlsx' # 경로(시청률 데이터)를 지정해준다.
filename
df_time = pd.read_excel(filename,header = 1,index_label = '시간대') # xlsx 읽기

In [ ]:
df_time.head

<bound method NDFrame.head of                      시간대  2019-01-01  ...  2019-12-31  2019-01-01 to 2019-12-31
0                  02:00       0.000  ...       0.000                     0.003
1                  02:01       0.000  ...       0.012                     0.003
2                  02:02       0.000  ...       0.000                     0.004
3                  02:03       0.000  ...       0.000                     0.004
4                  02:04       0.000  ...       0.000                     0.004
...                  ...         ...  ...         ...                       ...
1436               01:56       0.000  ...       0.000                     0.004
1437               01:57       0.000  ...       0.000                     0.004
1438               01:58       0.000  ...       0.000                     0.004
1439               01:59       0.000  ...       0.000                     0.004
1440  월화수목금토일02:00-01:59       0.004  ...       0.005                     0.004

[1441 row

In [ ]:
df_time=df_time.iloc[0:len(df_time)-1:,0:] # 마지막 부분 자르기

In [ ]:
df_time

,시간대,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,2019-01-11,2019-01-12,2019-01-13,2019-01-14,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-19,2019-01-20,2019-01-21,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,...,2019-11-23,2019-11-24,2019-11-25,2019-11-26,2019-11-27,2019-11-28,2019-11-29,2019-11-30,2019-12-01,2019-12-02,2019-12-03,2019-12-04,2019-12-05,2019-12-06,2019-12-07,2019-12-08,2019-12-09,2019-12-10,2019-12-11,2019-12-12,2019-12-13,2019-12-14,2019-12-15,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-21,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
0,02:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.000,0.006,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.038,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.048,0.0,0.0,0.0,0.000,0.018,0.000,0.035,0.030,0.016,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.012,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.003
1,02:01,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.005,0.000,0.0,0.0,0.000,0.0,0.000,0.006,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.038,0.000,...,0.0,0.0,0.0,0.0,0.000,0.022,0.000,0.0,0.0,0.000,0.048,0.0,0.0,0.0,0.000,0.018,0.000,0.035,0.030,0.016,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.012,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.012,0.003
2,02:02,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.005,0.000,0.0,0.0,0.000,0.0,0.000,0.006,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.038,0.000,...,0.0,0.0,0.0,0.0,0.000,0.022,0.000,0.0,0.0,0.000,0.033,0.0,0.0,0.0,0.000,0.018,0.018,0.035,0.067,0.016,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.012,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.004
3,02:03,0.0,0.0,0.014,0.0,0.000,0.0,0.0,0.0,0.000,0.005,0.000,0.0,0.0,0.000,0.0,0.000,0.006,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.038,0.000,...,0.0,0.0,0.0,0.0,0.000,0.022,0.000,0.0,0.0,0.000,0.033,0.0,0.0,0.0,0.000,0.018,0.018,0.035,0.067,0.016,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.012,0.0,0.0,0.000,0.000,0.017,0.0,0.000,0.0,0.0,0.000,0.000,0.004
4,02:04,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.005,0.000,0.0,0.0,0.000,0.0,0.000,0.006,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.038,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.012,0.033,0.0,0.0,0.0,0.000,0.018,0.000,0.035,0.067,0.016,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.012,0.0,0.0,0.000,0.000,0.017,0.0,0.000,0.0,0.0,0.000,0.000,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,01:55,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.004,0.000,0.000,0.0,0.0,0.000,0.0,0.005,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.021,0.0,0.0,0.0,0.0,0.0,0.0,0.039,0.000,0.032,...,0.0,0.0,0.0,0.0,0.022,0.000,0.013,0.0,0.0,0.033,0.000,0.0,0.0,0.0,0.058,0.000,0.036,0.031,0.000,0.000,0.0,0.000,0.0,0.0,0.009,0.0,0.0,0.036,0.0,0.0,0.013,0.000,0.000,0.0,0.015,0.0,0.0,0.000,0.000,0.004
1436,01:56,0.0,0.0,0.000,0.0,0.027,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.005,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.045,0.000,0.032,...,0.0,0.0,0.0,0.0,0.000,0.000,0.013,0.0,0.0,0.033,0.000,0.0,0.0,0.0,0.070,0.000,0.036,0.031,0.000,0.000,0.0,0.000,0.0,0.0,0.009,0.0,0.0,0.013,0.0,0.0,0.013,0.000,0.000,0.0,0.015,0.0,0.0,0.000,

In [ ]:
df['노출(분)'] # 노출분 가져오기 
exposure = []
for i in df['노출(분)']:
    exposure.append(i)

In [ ]:
merge_data=[]

for i in range(0,len(start_time)): # 지금 현재 8분정도 걸림
# for i in range(4300,4400): # 
    if start_time[i][0] == '2020-01-01':
         break
    day_data=[]
    day_data.append([start_time[i][0],start_time[i][1]+'~'+end_time[i][1]])
    for j in range(0,1440):
        if int(exposure[i])+1 == len(day_data):
            break
        if int(start_time[i][1].split(":")[0]) == int(df_time['시간대'][j].split(":")[0]): # 시작시간과 시청률의 시 가 같을때(ex 06:00 - 06:20)
            if int(end_time[i][1].split(":")[1]) == 0:  # end time 의 분이 0일때 
                if int(start_time[i][1].split(":")[1]) <= int(df_time['시간대'][j].split(":")[1]) < 60:  #start time ~ endtime(60으로 처리)분을 
                    if df_time[start_time[i][0]][j] == 0:       # 시청률이 0 일때 
                        day_data.append( [str(df_time['시간대'][j]),0.00025 ]) # 0대신 0.00025를 넣는다
                    else:
                        day_data.append( [str(df_time['시간대'][j]),df_time[start_time[i][0]][j] ])  #아닐때는 기존의 값을 넣는다.
            else: # # end time 의 분이 0이 아닐때는
                if int(start_time[i][1].split(":")[0]) != int(end_time[i][1].split(":")[0]): # (ex06:50 - 07:10) 시작 시간과 종료시간의 시 가 다르면 
                    if int(start_time[i][1].split(":")[1]) <= int(df_time['시간대'][j].split(":")[1]) < 60: #start time과 end time 사이의 값만 가져온다
                        if df_time[start_time[i][0]][j] == 0:  # 만약 시청률이 0이면 0대신 0.00025를 넣는다. 
                            day_data.append( [str(df_time['시간대'][j]),0.00025 ])
                        else:
                            day_data.append( [str(df_time['시간대'][j]),df_time[start_time[i][0]][j] ])
                else:
                    if int(start_time[i][1].split(":")[1]) <= int(df_time['시간대'][j].split(":")[1]) < int(end_time[i][1].split(":")[1]): #start time과 end time 사이의 값만 가져온다
                        if df_time[start_time[i][0]][j] == 0:  # 만약 시청률이 0이면 0대신 0.00025를 넣는다. 
                            day_data.append( [str(df_time['시간대'][j]),0.00025 ])
                        else:
                            day_data.append( [str(df_time['시간대'][j]),df_time[start_time[i][0]][j] ])
        elif int(end_time[i][1].split(":")[0]) == int(df_time['시간대'][j].split(":")[0]): # 끝나는시간의 시 가 시청률 시간의 시가 같을때 (ex06:50 - 07:10)
            if int(end_time[i][1].split(":")[1]) == 0:
                if 0 <= int(df_time['시간대'][j].split(":")[1]) < 60: # end_time이 0 분이면 60으로 처리한다
                    if df_time[start_time[i][0]][j] == 0:
                        day_data.append( [str(df_time['시간대'][j]),0.00025 ])
                    else:
                        day_data.append( [str(df_time['시간대'][j]),df_time[start_time[i][0]][j] ])
            else:
                if 0 <= int(df_time['시간대'][j].split(":")[1]) < int(end_time[i][1].split(":")[1]):# end_time이 0 분이 아니면 일반 값으로 가져온다.
                    if df_time[start_time[i][0]][j] == 0:
                        day_data.append( [str(df_time['시간대'][j]),0.00025 ])
                    else:
                        day_data.append( [str(df_time['시간대'][j]),df_time[start_time[i][0]][j] ])
    merge_data.append(day_data)


In [ ]:
len(merge_data)

20565

In [ ]:
value_lists = []
for i in range(0,len(merge_data)):
    value_list = []
    for j in range(1,len(merge_data[i])):
        value_list.append(float(merge_data[i][j][1]))
    # print("방송날짜 : {}".format(merge_data[i][0]))
    # print("평균 : {}".format( round(np.mean( value_list),5)))
    # print("중앙값 : {}".format( np.median( value_list)))
    # print("최고 시청률 : {}".format(np.max( value_list)))
    else:
        value_lists.append([merge_data[i][0][0].split("~")[0]+' '+merge_data[i][0][1].split("~")[0], str(round(np.mean( value_list),5)),str(np.median( value_list)),str(np.max( value_list))])

In [ ]:
len(value_lists)

20565

In [ ]:
value_lists[-1]

['2019-12-31 23:40', '0.02065', '0.024', '0.038']

In [ ]:
value_lists[0:10]

[['2019-01-01 6:00', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 6:20', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 6:40', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 7:00', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 7:20', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 7:40', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 8:00', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 8:20', '0.00025', '0.00025', '0.00025'],
 ['2019-01-01 8:40', '0.00199', '0.00025', '0.035'],
 ['2019-01-01 9:00', '0.00025', '0.00025', '0.00025']]

In [ ]:
import csv

In [ ]:
with open('rating_summary.csv', 'w',newline='') as file:
    writer = csv.writer(file)
    for i in value_lists:
        writer.writerow(i)